# Differential Expression Analysis with DEseq2
We have quantified the transcript expression using `Salmon` which 

In [1]:

#.libPaths('/home/ualtintas/miniconda3/envs/notebook/lib/R/library')
options(warn=-1)
options(repr.matrix.max.cols=100, repr.matrix.max.rows=10)

library(DESeq2)
library(edgeR)
library(tximport)
library(apeglm)
library(GenomicFeatures)


txdb <- makeTxDbFromGFF(file="/groups/lackgrp/genomeAnnotations/hg38/hg38.ncbiRefSeq.simple.gtf")
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME") 

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [7]:
fns = list.files("rnaseq-pipeline/results_hg38/salmon", pattern = "quant.sf", recursive = T, full.names = T)

names(fns) = basename(dirname(fns))

metaDataFile = "data/rna/metadata.tsv"
metaData = read.table(metaDataFile,header=TRUE)

In [3]:
fns

AR_rep1 
           "rnaseq-pipeline/results_hg38/salmon/AR_rep1/quant.sf" 
                                                          AR_rep2 
           "rnaseq-pipeline/results_hg38/salmon/AR_rep2/quant.sf" 
                                                          AR_rep3 
           "rnaseq-pipeline/results_hg38/salmon/AR_rep3/quant.sf" 
                                               GSM3502415_MR49F_1 
"rnaseq-pipeline/results_hg38/salmon/GSM3502415_MR49F_1/quant.sf" 
                                               GSM3502416_MR49F_2 
"rnaseq-pipeline/results_hg38/salmon/GSM3502416_MR49F_2/quant.sf" 
                                               GSM3502417_MR49F_3 
"rnaseq-pipeline/results_hg38/salmon/GSM3502417_MR49F_3/quant.sf" 
                                               GSM3502418_LNCaP_1 
"rnaseq-pipeline/results_hg38/salmon/GSM3502418_LNCaP_1/quant.sf" 
                                               GSM3502419_LNCaP_2 
"rnaseq-pipeline/results_hg38/salmon/GSM3502419_LNCaP_2/quant.sf" 
                                               GSM3502420_LNCaP_3 
"rnaseq-pipeline/results_hg38/salmon/GSM3502420_LNCaP_3/quant.sf" 
                                               GSM4777223_LNCaP_1 
"rnaseq-pipeline/results_hg38/salmon/GSM4777223_LNCaP_1/quant.sf" 
                                               GSM4777224_LNCaP_2 
"rnaseq-pipeline/results_hg38/salmon/GSM4777224_LNCaP_2/quant.sf" 
                                               GSM4777231_MR49F_1 
"rnaseq-pipeline/results_hg38/salmon/GSM4777231_MR49F_1/quant.sf" 
                                               GSM4777232_MR49F_2 
"rnaseq-pipeline/results_hg38/salmon/GSM4777232_MR49F_2/quant.sf" 
                                                          NT_rep1 
           "rnaseq-pipeline/results_hg38/salmon/NT_rep1/quant.sf" 
                                                          NT_rep2 
           "rnaseq-pipeline/results_hg38/salmon/NT_rep2/quant.sf" 
                                                          NT_rep3 
           "rnaseq-pipeline/results_hg38/salmon/NT_rep3/quant.sf" 
                                                  SMARCC2sg1_rep1 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg1_rep1/quant.sf" 
                                                  SMARCC2sg1_rep2 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg1_rep2/quant.sf" 
                                                  SMARCC2sg1_rep3 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg1_rep3/quant.sf" 
                                                  SMARCC2sg2_rep1 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg2_rep1/quant.sf" 
                                                  SMARCC2sg2_rep2 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg2_rep2/quant.sf" 
                                                  SMARCC2sg2_rep3 
   "rnaseq-pipeline/results_hg38/salmon/SMARCC2sg2_rep3/quant.sf"

In [3]:

idx = (metaData$Target %in% c('NT', 'AR')) & (metaData$Dataset == 'BG')

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "NT")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

resLFC = lfcShrink(dds, coef="Target_AR_vs_NT",type="apeglm")
summary(resLFC)

write.table(file="data/rna/deg/AR.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

3 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences


out of 19016 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 2482, 13%
LFC < 0 (down)     : 2421, 13%
outliers [1]       : 5, 0.026%
low counts [2]     : 2212, 12%
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [8]:
# TODO : metadata
idx = (metaData$Target %in% c('NT', 'SMARCC2sg1'))

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "NT")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)


resLFC = lfcShrink(dds, coef="Target_SMARCC2sg1_vs_NT",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/SMARCC2sg1.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)


reading in files with read_tsv

1 
2 
3 
4 
5 
6 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

4 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest



In [14]:

resLFC = lfcShrink(dds, coef="Target_SMARCC2sg1_vs_NT",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/SMARCC2sg1.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895




out of 18926 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 671, 3.5%
LFC < 0 (down)     : 559, 3%
outliers [1]       : 8, 0.042%
low counts [2]     : 3670, 19%
(mean count < 8)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [15]:
# TODO : metadata
idx = (metaData$Target %in% c('NT', 'SMARCC2sg2'))

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "NT")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

resLFC = lfcShrink(dds, coef="Target_SMARCC2sg2_vs_NT",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/SMARCC2sg2.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

reading in files with read_tsv

1 


2 
3 
4 
5 
6 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

5 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.o


out of 18944 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 595, 3.1%
LFC < 0 (down)     : 562, 3%
outliers [1]       : 6, 0.032%
low counts [2]     : 4407, 23%
(mean count < 12)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [24]:

idx = (metaData$Target %in% c('LNCaP', 'MR49F')) & (metaData$Dataset == 'GSE123379')

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "LNCaP")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

resLFC = lfcShrink(dds, coef="Target_MR49F_vs_LNCaP",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/GSE123379.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

8 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences


out of 16622 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 4148, 25%
LFC < 0 (down)     : 4043, 24%
outliers [1]       : 4, 0.024%
low counts [2]     : 0, 0%
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [34]:

idx = (metaData$Target %in% c('LNCaP', 'MR49F')) & (metaData$Dataset == 'GSE138460')

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "LNCaP")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

resLFC = lfcShrink(dds, coef="Target_MR49F_vs_LNCaP",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/GSE138460.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

reading in files with read_tsv

1 
2 
3 
4 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

7 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    


out of 17659 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 2052, 12%
LFC < 0 (down)     : 1970, 11%
outliers [1]       : 0, 0%
low counts [2]     : 343, 1.9%
(mean count < 3)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [30]:

idx = (metaData$Target %in% c('NT', 'AR', 'SMARCC2sg1', 'SMARCC2sg2'))

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "NT")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

write.table(file="data/rna/normalized.tsv", x=as.data.frame(counts(dds, normalized=TRUE)), sep="\t", quote = FALSE)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing



In [8]:

idx = (metaData$Target %in% c('NT', 'SMARCC2'))

txi = tximport(fns[idx], type = "salmon",tx2gene = tx2gene)

dds <- DESeqDataSetFromTximport(txi, metaData[idx,], ~Target)

keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

dds$Target = relevel(dds$Target, ref = "NT")

dds$Target <- droplevels(dds$Target)

dds <- DESeq(dds)

resLFC = lfcShrink(dds, coef="Target_SMARCC2_vs_NT",type="apeglm")
summary(resLFC)


write.table(file="data/rna/deg/SMARCC2.tsv", x=as.data.frame(resLFC), sep="\t", quote = FALSE)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 


summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

2 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large di


out of 19927 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 1141, 5.7%
LFC < 0 (down)     : 1151, 5.8%
outliers [1]       : 15, 0.075%
low counts [2]     : 4243, 21%
(mean count < 7)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [6]:
idx

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
[25]  TRUE  TRUE  TRUE  TRUE